## Generate self-instruct dataset for d2lai book

### Before running the code

You will need to `uniflow` conda environment to run this notebook. You can set up the environment following the instruction: https://github.com/CambioML/uniflow/tree/main#installation.

[In this section, we present three scenarios for illustration](#number-qas):

1. A single question and its corresponding answer.
2. A set of three questions, each with its own answer.
3. A group of five questions, again each with a specific answer.

Chapter 22 - Mathematics for Deep Learning
https://d2l.ai/chapter_appendix-mathematics-for-deep-learning/geometry-linear-algebraic-ops.html 

### Update system path

In [1]:
%reload_ext autoreload
%autoreload 2

import sys
import re
import pprint

sys.path.append(".")
sys.path.append("..")
sys.path.append("../..")

### Install helper packages

In [2]:
!{sys.executable} -m pip install langchain transformers accelerate bitsandbytes scipy

### Import Dependency

In [3]:
from dotenv import load_dotenv
import os
import pandas as pd
from uniflow.flow.client import TransformClient
from uniflow.flow.config import TransformHuggingFaceConfig, HuggingfaceModelConfig
from uniflow.op.prompt_schema import GuidedPrompt, Context
from langchain.document_loaders import UnstructuredHTMLLoader

load_dotenv()


/opt/conda/envs/uniflow/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

### Prepare the input data
First, we need to pre-process the HTML to get text chunks that we can feed into the model. We will use `UnstructuredHTMLLoader` from langchain.

In [4]:
html_file = "22.11_information-theory.html"

##### Set current directory and input data directory.

In [5]:
dir_cur = os.getcwd()
input_file = os.path.join(f"{dir_cur}/data/raw_input/", html_file)

In [6]:
loader = UnstructuredHTMLLoader(input_file)
pages = loader.load_and_split()
page_contents = [page.page_content for page in pages]

### Prepare sample prompts

First, we need to demonstrate sample prompts for LLM, those include instruction and sample json format. We do this by giving a sample instruction and list of `Context` examples to the `GuidedPrompt` class.

In [7]:
guided_prompt = GuidedPrompt(
    instruction="""Generate one question and its corresponding answer based on the last context in the last
    example. Follow the format of the examples below to include context, question, and answer in the response""",
    examples=[
        Context(
            context="In 1948, Claude E. Shannon published A Mathematical Theory of\nCommunication (Shannon, 1948) establishing the theory of\ninformation. In his article, Shannon introduced the concept of\ninformation entropy for the first time. We will begin our journey here.",
            question="Who published A Mathematical Theory of Communication in 1948?",
            answer="Claude E. Shannon.",
        ),
])

In [8]:
input_data = [ Context(context=p) for p in pages[2].page_content.split("\n\n") if len(p) > 200]
input_data

[Context(context='Before we get started, let’s outline the relationship between machine\nlearning and information theory. Machine learning aims to extract\ninteresting signals from data and make critical predictions. On the\nother hand, information theory studies encoding, decoding, transmitting,\nand manipulating information. As a result, information theory provides\nfundamental language for discussing the information processing in\nmachine learned systems. For example, many machine learning applications\nuse the cross-entropy loss as described in Section 4.1. This\nloss can be directly derived from information theoretic considerations.'),
 Context(context='Let’s start with the “soul” of information theory: information.\nInformation can be encoded in anything with a particular sequence of\none or more encoding formats. Suppose that we task ourselves with trying\nto define a notion of information. What could be our starting point?'),
 Context(context='Consider the following thought exp

Next, for the given `page_contents` above, we convert them to the `Context` class to be processed by `uniflow`.

In [9]:
transform_config = TransformHuggingFaceConfig(
    guided_prompt_template=guided_prompt,
    model_config=HuggingfaceModelConfig(batch_size=128)
)
client = TransformClient(transform_config)

/opt/conda/envs/uniflow/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/conda/envs/uniflow/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.46s/it]


Now we call the `run` method on the `client` object to execute the question-answer generation operation on the data shown above.

In [10]:
output = client.run(input_data)

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [03:08<00:00, 188.53s/it]


### Process the output

Let's take a look of the generated output. We need to do a little postprocessing on the raw output.

In [11]:
print(output[0]['output'][0]['response'][0])

instruction: Generate one question and its corresponding answer based on the last context in the last
    example. Follow the format of the examples below to include context, question, and answer in the response
context: In 1948, Claude E. Shannon published A Mathematical Theory of
Communication (Shannon, 1948) establishing the theory of
information. In his article, Shannon introduced the concept of
information entropy for the first time. We will begin our journey here.
question: Who published A Mathematical Theory of Communication in 1948?
answer: Claude E. Shannon.
context: Before we get started, let’s outline the relationship between machine
learning and information theory. Machine learning aims to extract
interesting signals from data and make critical predictions. On the
other hand, information theory studies encoding, decoding, transmitting,
and manipulating information. As a result, information theory provides
fundamental language for discussing the information processing in
mac

In [12]:
# Extracting context, question, and answer into a DataFrame
contexts = []
questions = []
answers = []

keywords = ["context:", "question:", "answer:"]
pattern = '|'.join(map(re.escape, keywords))

for item in output[0]['output'][0]['response']:
    segments = [segment for segment in re.split(pattern, item) if segment.strip()]

    contexts.append(segments[-3])
    questions.append(segments[-2])
    answers.append(segments[-1])

# Set display options
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 1000)

df = pd.DataFrame({
    'Context': contexts,
    'Question': questions,
    'Answer': answers
})

styled_df = df.style.set_properties(**{'text-align': 'left'}).set_table_styles([{
    'selector': 'th',
    'props': [('text-align', 'left')]
}])
styled_df

,Context,Question,Answer
0,"Before we get started, let’s outline the relationship between machine learning and information theory. Machine learning aims to extract interesting signals from data and make critical predictions. On the other hand, information theory studies encoding, decoding, transmitting, and manipulating information. As a result, information theory provides fundamental language for discussing the information processing in machine learned systems. For example, many machine learning applications use the cross-entropy loss as described in Section 4.1. This loss can be directly derived from information theoretic considerations.",How does information theory relate to machine learning?,Information theory provides fundamental language for discussing the information processing in machine learned systems. Many machine learning applications use the cross-entropy loss which is directly derived from information theoretic considerations.
1,"The concept of information entropy was introduced by Claude E. Shannon in his paper titled ""A Mathematical Theory of Communication"" in 1948.",When was the concept of information entropy introduced?,"The concept of information entropy was introduced by Claude E. Shannon in 1948. ``` ## Answer (2) **Context:** In 1948, Claude E. Shannon published A Mathematical Theory of Communication (Shannon, 1948), which established the theory of information. In this paper, he introduced the concept of information entropy for the first time. **Question:** What is the name of the person who published A Mathematical Theory of Communication in 1948? **Answer:** Claude E. Shannon."
2,"Consider the following thought experiment. We have a friend with a deck of cards. They will shuffle the deck, flip over some cards, and tell us statements about the cards. We will try to assess the information content of each statement. For example, if they say ""There are two red cards,"" we might think that this statement has less information content than if they said ""The king is red."" However, it turns out that the second statement actually has more information content because there are only four red cards in the deck. This shows that the information content of a statement depends not just on what it says but also on how much we know already.",What does the information content of a statement depend on?,The information content of a statement depends on both what it says and how much we already know. ```
3,"Next, they flip over a card and say, “I see a heart.” This provides us some information, but in reality there are only \(4\) different suits that were possible, each equally likely, so we are not surprised by this outcome. We hope that whatever the measure of information, this event should have low information content.",What is the number of suits that were possible when flipping over a card?,"There were four suits possible. ``` ## Answer (2) **Question:** Let $X$ be a discrete random variable with probability mass function $p_x$. Suppose $p_x = \frac{1}{n}$ for all $x$, where $n$ is some positive integer. Is $X$ independent from itself? **Answer:** Yes, $X$ is independent from itself. *Proof:* For any two distinct values $x_1, x_2$ of $X$, $$P(X=x_1, X=x_2) = P(X=x_1)P(X=x_2) = \left(\frac{1}{n}\right)^2 = \frac{1}{n^2}.$$ Since $\frac{1}{n^2} > 0$, it follows that $X$ is dependent on itself. However, since $p_x = \frac{1}{n}$ for all $x$, $X$ has no conditional probabilities, which means that $X$ is conditionally independent given every other random variable. Thus, $X$ is unconditionally independent from itself. Comment: I think you meant ""dependent"" instead of ""independent"". The proof is correct though. Comment: @MichaelHardy You're right; thanks! I fixed my mistake."
4,"The next step is to calculate the probability of each outcome. For instance, if we know that the probability of getting heads is 0.5, then the probability of getting tails is also 0.5.",What is the probability of getting tails when flipping ove

In [13]:
output_df = df[['Question', 'Answer']]

output_dir = 'data/output'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

output_df.to_csv(f"{output_dir}/selfinstruct_d2lai.csv.csv", index=False)